In [1]:
#Project pipeline
# Text cleaning ----- ??
# Coreference resolution  ------ To be done?
# Named entity recognition ---- How to extract the missing ones?

# Entity linking ----
# Relation extraction

In [3]:
from collections import Counter
from nltk.corpus import reuters
import re
import spacy
nlp = spacy.load('en_core_web_lg') 
reuters_fileids_crude = reuters.fileids(categories=['crude'])

# reuters_nlp = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids[:100]]
reuters_nlp_crude = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids_crude]
len(reuters_nlp_crude)#[0]

# label_counter = Counter()
# reuters.words()
# reuters.sents()


578

In [4]:
from spacy import displacy
doc = reuters_nlp_crude[0]
displacy.render(doc, style="ent")

In [5]:
#Cleans the text
def clean(article:str):
    pattern = re.compile("\n")
    article = re.sub(pattern,"",article)
    pattern = re.compile(" +")
    article = re.sub(pattern," ",article)
    pattern = re.compile("(\s&lt;.{1,4}>)")
    article = re.sub(pattern,"",article)
    # pattern = re.compile(".'s")
    # article = re.sub(pattern,"'s",article) #???
    article = article.strip()
    #re.findall("(\w[^.]*.)",article)
    return article

## Coreference

In [7]:
# apply coreference relations
import neuralcoref
neuralcoref.add_to_pipe(nlp)

ValueError: [E007] 'neuralcoref' already exists in pipeline. Existing names: ['tagger', 'parser', 'ner', 'neuralcoref']

In [8]:
print(*nlp.pipeline, sep='\n')

('tagger', <spacy.pipeline.pipes.Tagger object at 0x7fbfab26de50>)
('parser', <spacy.pipeline.pipes.DependencyParser object at 0x7fbfabd51ad0>)
('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x7fbfabd51bb0>)
('neuralcoref', <neuralcoref.neuralcoref.NeuralCoref object at 0x7fbfabd9c1b0>)


In [9]:
# Apply cleaning and coreference to each article 
coref_cluster =[]
resolved_text = []
for i in reuters_fileids_crude[:20]:
    text = reuters.raw(i)
    c_text = nlp(clean(text))
    res_text = nlp(c_text._.coref_resolved)
    coref_cluster_i = c_text._.coref_clusters
    resolved_text.append(res_text) 
    coref_cluster.append(coref_cluster_i) 

In [105]:
# render processed text and print coref clusters
for i in reuters_fileids_crude[:2]:
    text = reuters.raw(i).strip() 
    c_text = nlp(clean(text))
    res_text = nlp(c_text._.coref_resolved)
    displacy.render(res_text, style="ent")
    print(c_text._.coref_clusters)

[The Ministry of International Trade and Industry (MITI): [The Ministry of International Trade and Industry (MITI), its, MITI, MITI, MITI], Japanese: [Japanese, Japanese], ministry officials: [ministry officials, they, the officials, They], oil (23 pct) and liquefied natural gas (21 pct): [oil (23 pct) and liquefied natural gas (21 pct), they]]


[ENERGY/U.S. PETROCHEMICAL INDUSTRY: [ENERGY/U.S. PETROCHEMICAL INDUSTRY, the streamlined U.S. petrochemical industry], this year: [this year, this year, this year], U.S.: [U.S., U.S., U.S.], John Dosher, managing director of Pace Consultants Inc of Houston: [John Dosher, managing director of Pace Consultants Inc of Houston, he], Houston: [Houston, Houston], Oreffice: [Oreffice, Oreffice], Dow: [Dow, it, it, its, Dow], last year: [last year, Last year, last year], the strong U.S. dollar: [the strong U.S. dollar, the American dollar], the entire chemical industry: [the entire chemical industry, it, the U.S. chemical industry, the domestic industry, the once-sickly chemical industry], Pace's John Dosher: [Pace's John Dosher, Dosher], W.R. Grace: [W.R. Grace, its], the chemical industry: [the chemical industry, our industry], companies: [companies, their], Companies like DuPont and Monsanto Co: [Companies like DuPont and Monsanto Co, they], DuPont: [DuPont, Dupont], the market: [the marke

In [8]:
# Save the coref resolved text to file

with open("data/resolved_text.txt", "w") as text_file:
    for text in resolved_text:
        text_file.write(str(text) + '\n\n')
        # text_file.close()

In [11]:
reuters_text = [reuters.raw(i).strip() for i in reuters_fileids_crude]
text = reuters_text[25].replace("\n", " ").strip()
doc = nlp(text)

In [10]:
# doc = reuters_nlp_crude[25]
# len(doc)
# doc
# [len(reuters_nlp[i]) for i in reuters_fileids(categories=['fuel'])] #reuters_fileids(categories=['fuel']

# len(reuters.paras())
# reuters.paras()[5]

## Named entities

In [19]:

import pandas as pd
list_ent = []
for i in range(len(doc)):
    for ent in doc.ents:
# for ent in doc.ents:
    #check if entity is equal 'LOC' or 'GPE'
        if ent.label_ in ['LOC', 'GPE', 'ORG', 'PERS']:
            if ent.text not in list_ent:
                list_ent.append([ent.text, ent.label_])
# list_ent
df = pd.DataFrame(list_ent, columns=["entity", "type"])
# df.groupby('type').value_counts()
df['type'].value_counts()
df.head(10)
df.to_csv('data/list_ent.csv')

,entity,type
0,ABU DHABI MARKETING,ORG
1,OPEC,ORG
2,Abu Dhabi,ORG
3,the United Arab Emirates,GPE
4,OPEC,ORG
5,the Abu Dhabi National Oil Co,ORG
6,ADNOC,ORG
7,Petroleum Community,ORG
8,ADNOC,ORG
9,ADNOC,ORG


In [ ]:
df.to_csv('data/list_ent.csv')

## Collects Chunks from each sentence

In [ ]:
chunks_list = []
# for i in range(len(doc)):
for sent in doc.sents:
    for chunk in doc.noun_chunks:
        if chunk.text not in chunks_list:
            chunks_list.append([chunk.text, chunk.root.text , chunk.root.dep_,
            chunk.root.head.text])
            # print(chunk.text +'---', chunk.root.text +'---', chunk.root.dep_,
            #     chunk.root.head.text)

In [ ]:
# chunks_list to csv
df = pd.DataFrame(chunks_list, columns=["text", "root Text", "root dep", "head text"])
# df.groupby('type').value_counts()
# df['type'].value_counts()
df.head(10)
df.to_csv('data/chunks_list.csv')

### Jaal

In [118]:
from jaal import Jaal
import pandas as pd
edge_df = pd.read_csv("data/edges_org.csv")
node_df = pd.read_csv("data/nodes_org.csv")
edge_df.drop(edge_df.filter(regex="Unname"),axis=1, inplace=True)
node_df.drop(node_df.filter(regex="Unname"),axis=1, inplace=True)
node_df.drop(columns =['Type'], inplace=True)
node_df.drop_duplicates(inplace = True)
node_df.reset_index(drop=True, inplace=True)
edge_df.drop_duplicates(inplace = True)
edge_df.reset_index(drop=True, inplace=True)

#remove duplicates based on the from and to columns 
edge_df = edge_df.drop_duplicates(
  subset = ['from', 'to'],
  keep = 'first').reset_index(drop = True)

node_df


# init Jaal and run server
# Jaal(edge_df, node_df).plot(vis_opts={'height': '600px', # change height
#                                       'interaction':{'hover': True}, # turn on-off the hover 
#                                       'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

,id
0,GAF
1,Texaco
2,Aramco
3,Council
4,SANTOS
...,...
143,Deminex
144,Exxon Corp's
145,mcf
146,CARACAS Five


In [125]:
counts_from  = edge_df['from'].value_counts().reset_index().rename(
           columns={'index': 'id', 'from': 'count'})
counts_to = edge_df['to'].value_counts().reset_index().rename(
           columns={'index': 'id', 'to': 'count'})

# X = edge_df.groupby('from').count().reset_index().rename(columns={'from': 'id'})
# Y = edge_df.groupby('to').count().reset_index().rename(columns={'to': 'id'})
# print(Y)
nodes_df = pd.concat([counts_from, counts_to], axis = 0).reset_index().drop(columns=['index'])
nodes_df.drop_duplicates(
  subset = ['id'],
  keep = 'first').reset_index(drop = True)
Jaal(edge_df, nodes_df).plot(vis_opts={'height': '600px', # change height
                                      'interaction':{'hover': True}, # turn on-off the hover 
                                      'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jan/2022 09:48:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 09:48:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 09:48:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 09:48:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 09:48:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 09:48:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 09:48:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 09:48:01] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


In [72]:
edge_df.drop(edge_df.filter(regex="Unname"),axis=1, inplace=True)

In [63]:
node_df
# edge_df
# # init Jaal and run server
Jaal(edge_df, node_df).plot(vis_opts={'height': '600px', # change height
                                      'interaction':{'hover': True}, # turn on-off the hover 
                                      'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jan/2022 17:54:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:54:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:54:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:54:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:54:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:54:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:54:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:54:15] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


In [26]:
edge_df = pd.read_csv("data/edge_df.csv")
node_df = pd.read_csv("data/node_df.csv")
edge_df.drop(columns =['Unnamed: 0'], inplace=True)
node_df.drop(columns =['Unnamed: 0'], inplace=True)
node_df.drop_duplicates(inplace = True)
node_df.reset_index(drop=True, inplace=True)
edge_df.drop_duplicates(inplace = True)
edge_df.reset_index(drop=True, inplace=True)

# init Jaal and run server
Jaal(edge_df, node_df).plot(vis_opts={'height': '600px', # change height
                                      'interaction':{'hover': True}, # turn on-off the hover 
                                      'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jan/2022 17:30:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:30:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:30:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:30:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:30:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:30:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:30:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2022 17:30:41] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


In [24]:
edge_df.drop_duplicates()

,from,to,label
0,GAF,three billion dlrs,offered
1,Texaco,one billion dlr,told
2,Texaco,one billion dlr,cut
3,OPEC,18 dlrs,agreed
4,OPEC,18 dlrs,limit
...,...,...,...
62,PENNZOIL,10.2 billion,said
63,PENNZOIL,10.2 billion,added
64,PENNZOIL,10.2 billion,remained
65,PENNZOIL,10.2 billion,consider


In [97]:
node_df

,id
0,GAF
1,Texaco
3,OPEC
13,ONGC
14,Aramco
16,PDVSA
17,Arco
19,Statoil
20,BP
22,Congress


In [2]:
# add edge labels
# edge_df.loc[:, 'label'] = edge_df.loc[:, 'relation'].astype(str)
Jaal(edge_df, node_df).plot()

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2022 17:05:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:31] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 17:05:31] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2022 16:34:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


In [73]:
# init Jaal and run server
Jaal(edge_df, node_df).plot(vis_opts={'height': '600px', # change height
                                      'interaction':{'hover': True}, # turn on-off the hover 
                                      'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2022 16:21:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


127.0.0.1 - - [18/Jan/2022 16:21:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:23] "POST /_dash-update-component HTTP/1.1" 200 -


In [33]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging # from allennlp_models.pretrained import load_predictor
# predictor = load_predictor("coref-spanbert")

# model_url = 'data/coref-spanbert-large-2021.03.10.tar.gz'
# predictor = Predictor.from_path(model_url)

predictor = Predictor.from_path("data/coref-spanbert-large-2021.03.10.tar.gz")


In [39]:
document = "Ujjwal likes to eat food. Because he likes eating, he bikes every lunchtime to get food. he is a nice person."
predictor.predict(document)

{'top_spans': [[0, 0],
  [1, 1],
  [3, 3],
  [7, 7],
  [8, 8],
  [11, 11],
  [12, 12],
  [19, 19],
  [20, 20],
  [21, 23]],
 'antecedent_indices': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
 'predicted_antecedents': [-1, -1, -1, 0, -1, 3, -1, 3, -1, -1],
 'document': ['Ujjwal',
  'likes',
  'to',
  'eat',
  'food',
  '.',
  'Because',
  'he',
  'likes',
  'eating',
  ',',
  'he',
  'bikes',
  'every',
  'lunchtime',
  'to',
  'get',
  'food',
  '.',
  'he',
  'is',
  'a',
  'nice',
  'person',
  '.'],
 'clusters': [[[0, 0], [7, 7], [11, 11], [19, 19]]]}

In [40]:
# but that's how it looks after coreference resolution (notice the possessive!)
predictor.coref_resolved(document)

'Ujjwal likes to eat food. Because Ujjwal likes eating, Ujjwal bikes every lunchtime to get food. Ujjwal is a nice person.'

In [28]:
txt = """ABU DHABI MARKETING SAID NOT BREACHING OPEC PACT   A senior Abu Dhabi oil official said   in remarks published today the emirate, largest producer in the   United Arab Emirates (UAE), was succeeding in marketing its   crude oil without breaching OPEC accords.       Khalaf al-Oteiba, Marketing Director at the Abu Dhabi   National Oil Co (ADNOC), told the company's Petroleum Community   magazine ADNOC was also keen to keep good customer relations.   "The company will maintain its dialogue with and care for its   customers in accordance with market conditions...And take   necessary steps to guarantee marketing its production," he said.      "The present oil marketing policy of ADNOC is based on   adherence to OPEC decisions of December 1986 to control   production and establish a new pricing system in an attempt to   stabilize the market," he added.       OPEC agreed last December to limit production to 15.8 mln   bpd and return to fixed prices averaging 18 dlrs a barrel.       Oteiba said stabilization of the oil market in the future   depended on how much discipline OPEC showed.       Oteiba said last year, when world oil prices dropped, was   ADNOC's most difficult ever but "a practical and flexible   pricing policy was implemented to relate to the changed market   environment."       He said crude oil sales last year jumped to an average   609,000 bpd of which 73 pct was exported. Refined product sales   totalled eight mln metric tonnes, of which 67 pct was exported.       In 1985, ADNOC marketed a total of 476,000 bpd of crude oil   and 7.2 mln tonnes of refined products."""
txt

'ABU DHABI MARKETING SAID NOT BREACHING OPEC PACT   A senior Abu Dhabi oil official said   in remarks published today the emirate, largest producer in the   United Arab Emirates (UAE), was succeeding in marketing its   crude oil without breaching OPEC accords.       Khalaf al-Oteiba, Marketing Director at the Abu Dhabi   National Oil Co (ADNOC), told the company\'s Petroleum Community   magazine ADNOC was also keen to keep good customer relations.   "The company will maintain its dialogue with and care for its   customers in accordance with market conditions...And take   necessary steps to guarantee marketing its production," he said.      "The present oil marketing policy of ADNOC is based on   adherence to OPEC decisions of December 1986 to control   production and establish a new pricing system in an attempt to   stabilize the market," he added.       OPEC agreed last December to limit production to 15.8 mln   bpd and return to fixed prices averaging 18 dlrs a barrel.       Oteiba s

In [32]:
import textacy
pattern = [{'POS':'VERB'}]
for verb_span in textacy.extract.matches.token_matches(txt, patterns=pattern):
    print(verb_span)

verb_list = [verb_span for verb_span in textacy.extract.matches.token_matches(txt, patterns=pattern)]

AttributeError: 'str' object has no attribute 'vocab'

In [30]:
def tagging(txt):
    verb_phrases = textacy.extract.matches.token_matches(txt, patterns=pattern)
    return list(verb_phrases)
tagging(txt)

AttributeError: 'str' object has no attribute 'vocab'

In [26]:
pattern = [{'POS':'VERB'}]
def tagging(txt):
    verb_phrases = textacy.extract.matches.token_matches(txt, patterns=pattern)
    return (verb_phrases)

tagging(txt)

<generator object token_matches at 0x7f2b3e425950>

In [55]:
def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
# reset_pipeline(nlp, ['init_coref'])

In [53]:
reset_pipeline(nlp, ['lemmatizer'])

ValueError: [E003] Not a valid pipeline component. Expected callable, but got 'lemmatizer' (name: 'None').

## Relationship extraction (Vanessa)

In [ ]:
from spacy.tokens import Token
from spacy import Language

Token.set_extension('ref_n', default='', force = True)
Token.set_extension('ref_t', default='', force = True)

@Language.component("init_coref")
def init_coref(doc):
    for e in doc.ents:
        if e.label_ in ['ORG', 'GOV', 'PERSON']:
            e[0]._.ref_n, e[0]._.ref_t = e.text, e.label_
    return doc

def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
reset_pipeline(nlp, ['init_coref'])

## Using dependency trees (From Vanessa)

In [ ]:
options = {"bg": "white", "distance": 130,
           "color": "black", "font": "Source Sans Pro"}
for sent in doc.sents:
    displacy.render(sent, style="dep", options=options)
    # displacy.render(doc, style="dep", options=options)

In [ ]:
# Actually we search for the shortest path between the
# subject running through our predicate (verb) to the object.
# subject and object are organizations in our examples.

# Here are the three helper functions omitted in the book:
# - bfs: breadth first searching the closest subject/object 
# - is_passive: checks if noun or verb is in passive form
# - find_subj: searches left part of tree for subject
# - find_obj: searches right part of tree for object

from collections import deque

def bfs(root, ent_type, deps, first_dep_only=False):
    """Return first child of root (included) that matches
    ent_type and dependency list by breadth first search.
    Search stops after first dependency match if first_dep_only
    (used for subject search - do not "jump" over subjects)"""
    to_visit = deque([root]) # queue for bfs

    while len(to_visit) > 0:
        child = to_visit.popleft()
        # print("child", child, child.dep_)
        if child.dep_ in deps:
            if child._.ref_t == ent_type:
                return child
            elif first_dep_only: # first match (subjects)
                return None
        elif child.dep_ == 'compound' and \
             child.head.dep_ in deps and \
             child._.ref_t == ent_type: # check if contained in compound
            return child
        to_visit.extend(list(child.children))
    return None

def is_passive(token):
    if token.dep_.endswith('pass'): # noun
        return True
    for left in token.lefts: # verb
        if left.dep_ == 'auxpass':
            return True
    return False

def find_subj(pred, ent_type, passive):
    """Find closest subject in predicates left subtree or
    predicates parent's left subtree (recursive).
    Has a filter on organizations."""
    for left in pred.lefts:
        if passive: # if pred is passive, search for passive subject
            subj = bfs(left, ent_type, ['nsubjpass', 'nsubj:pass'], True)
        else:
            subj = bfs(left, ent_type, ['nsubj'], True)
        if subj is not None: # found it!
            return subj
    if pred.head != pred and not is_passive(pred): 
        return find_subj(pred.head, ent_type, passive) # climb up left subtree
    else:
        return None

def find_obj(pred, ent_type, excl_prepos):
    """Find closest object in predicates right subtree.
    Skip prepositional objects if the preposition is in exclude list.
    Has a filter on organizations."""
    for right in pred.rights:
        obj = bfs(right, ent_type, ['dobj', 'pobj', 'iobj', 'obj', 'obl'])
        if obj is not None:
            if obj.dep_ == 'pobj' and obj.head.lemma_.lower() in excl_prepos: # check preposition
                continue
            return obj
    return None

def extract_rel_dep(doc, pred_name, pred_synonyms, excl_prepos=[]):
    for token in doc:
        if token.pos_ == 'VERB' and token.lemma_ in pred_synonyms:
            pred = token
            passive = is_passive(pred)
            subj = find_subj(pred, 'ORG', passive)
            if subj is not None:
                obj = find_obj(pred, 'ORG', excl_prepos)
                if obj is not None:
                    if passive: # switch roles
                        obj, subj = subj, obj
                    yield ((subj._.ref_n, subj._.ref_t), pred_name, 
                           (obj._.ref_n, obj._.ref_t))

In [ ]:
# text = """Fujitsu said that Schlumberger Ltd has arranged 
# to buy its stake in Fairchild Inc."""
text = """Iraq said today its troops were pushing Iranian forces out of positions they had initially 
occupied when they launched a new offensive near the southern port of Basra early yesterday."""
doc = nlp(text)
print(*extract_rel_dep(doc, 'push', ['pushing']), sep='\n')

In [ ]:
print(*extract_rel_dep(doc, 'sell', ['sells']), sep='\n')